# Import libraries & UDFs
## Import essentials

In [50]:
from splinter import Browser
from bs4 import BeautifulSoup as bs

import caffeine
import time
from tqdm import tqdm

import pandas as pd

import requests
from bs4 import BeautifulSoup
import json
from flatten_json import flatten

## User defined functions

### Generate tag string

In [51]:
def tag_string(tag, html_element):
    typ = html_element.split("=")[0]
    name = html_element.split("=")[1]
    return tag + f"[{typ}=" + "\"" + name + "\"]"

### Display always on

For when some cell take too long to run

In [52]:
def on(): 
    caffeine.on(display=True)

### Launch driver

In [27]:
def launch_driver():
    global browser
    
    path = {"executable_path": "/Users/marclamy/Desktop/code/chromedriver6"}

    # Starting driver. Headless displays the browser. 
    browser = Browser("chrome", **path, headless=False) 
    
    # Let's go the the results page
    browser.visit("https://www.premierleague.com/results?co=1&se=1&cl=-1")

#     # Thanks for the cookies I take them all 🍪
#     browser.find_by_tag(tag_string("div", "class=btn-primary cookies-notice-accept")).click()

#     # There is an ad that is only temporary 
#     browser.find_by_tag(tag_string("a", "id=advertClose")).click()

# Get data

To get each match, I will scrape the Premier League's website which is made as follow for each game.

*premierleague.com / match /* ***{id of the game}***

First, I need the id of each game so I can loop from the list after. 


## Get season_id 

To scrape the ids of each game, I'll first need [chrome webdriver](https://chromedriver.chromium.org/downloads).

### Start the driver

In [20]:
launch_driver()

### Accept the cookies and close the ad 

### Get each season id 

I choose from 95 as until then there were 22 teams (82 more games per year) and some seasons were incomplete, not all match were listed. 

In [28]:
all_seasons = []

for x in range(1992, 2021):
    second = f"0{x+1}" if x+1 < 10 else x+1
    all_seasons.append(f"{x}/{str(x+1)[2:]}")

In [37]:
# launch_driver()

df_season_ids = pd.DataFrame(columns=["season", "season_id"])

browser.find_by_tag(tag_string("div", "data-dropdown-current=compSeasons"), wait_time=10).click()


for element in browser.find_by_tag("li"): 
    season_id = element["data-option-id"]
    season = element["data-option-name"]
    if season_id != None and season in all_seasons:
        df_season_ids.loc[len(df_season_ids)] = [season, season_id]

df_season_ids["nb_games"] = 380
df_season_ids.loc[[26, 27, 28], "nb_games"] = 462
df_season_ids

,season,season_id,nb_games
0,2020/21,363,380
1,2019/20,274,380
2,2018/19,210,380
3,2017/18,79,380
4,2016/17,54,380
5,2015/16,42,380
6,2014/15,27,380
7,2013/14,22,380
8,2012/13,21,380
9,2011/12,20,380


### Save season_id in a csv

In [38]:
df_season_ids = df_season_ids.sort_values(by="season").reset_index(drop=True)
df_season_ids.to_csv("real_data/season_ids.csv", index=False)

## Get match_id

In [40]:
on()

df_season_ids = pd.read_csv("real_data/season_ids.csv")
df_match_ids = pd.DataFrame(columns=["season", "season_id", "match_id", "nb_games"])

launch_driver()

for index, season_id in enumerate(df_season_ids["season_id"]):
    all_match_ids = set() # for unique values directly
    browser.visit(f"https://www.premierleague.com/results?co=1&se={season_id}&cl=-1")
    
    # There is a total of 380 games per season
    count = 0
    seasson_games_count = df_season_ids.loc[index, "nb_games"]
    while len(all_match_ids) != seasson_games_count: 
        tag = browser.find_by_tag("li", wait_time=10)
        all_match_ids = {e["data-comp-match-item"] for e in tag if e["data-comp-match-item"] != None} 
        
        browser.execute_script(f"window.scroll(0, document.body.scrollHeight)")
        
        if count % 2 == 0: 
            browser.execute_script(f"window.scroll(0, 0)")
            
            
        print(df_season_ids.loc[index, "season"], len(all_match_ids), count, " "*20, end="\r")
        
        count += 1
        
        if count == 40:
            all_match_ids = set()
            browser.visit(f"https://www.premierleague.com/results?co=1&se={season_id}&cl=-1")
            count = 0
        
        
    for match_id in all_match_ids:
        df_match_ids.loc[len(df_match_ids)] = [df_season_ids.loc[index, "season"], season_id, 
                                               match_id, seasson_games_count]
    
    

browser.quit()

In [42]:
print(df_match_ids.shape)
display(df_match_ids.head())

df_match_ids.to_csv("real_data/match_ids.csv", index=False)

(11266, 4)


,season,season_id,match_id,nb_games
0,1992/93,1,147,462
1,1992/93,1,134,462
2,1992/93,1,85,462
3,1992/93,1,289,462
4,1992/93,1,19,462


In [44]:
df_match_ids["match_id"]

0          147
1          134
2           85
3          289
4           19
         ...  
11261    59146
11262    59274
11263    59008
11264    58935
11265    59023
Name: match_id, Length: 11266, dtype: object

In [22]:
import pandas as pd
df_match_ids = pd.read_csv("real_data/match_ids.csv")

df_players = pd.read_csv('real_data/all_players_r.csv')
df_events = pd.read_csv('real_data/all_events_r.csv')
df_games = pd.read_csv('real_data/all_games_r.csv')

# df_players = pd.DataFrame()
# df_events = pd.DataFrame()
# df_games = pd.DataFrame()

manquants = set(df_match_ids["match_id"]) - set(df_players["match_id"])


## Get matchs

In [57]:
on()
df_match_ids = pd.read_csv("real_data/match_ids.csv")

# df_match_ids = pd.read_csv("real_data/match_ids.csv")

df_players = pd.read_csv('real_data/all_players_92.csv')
df_events = pd.read_csv('real_data/all_events_92.csv')
df_games = pd.read_csv('real_data/all_games_92.csv')

# df_players = pd.DataFrame()
# df_events = pd.DataFrame()
# df_games = pd.DataFrame()

# old = df_match_ids["match_id"]
# new = df_players["match_id"]

# ids = set(old) - set(new)


# ids = df_match_ids["match_id"]
manquants = set(df_match_ids["match_id"]) - set(df_games["match_id"])


print(len(manquants))
# print(fghjds)



# the code is running only the first 400 ids right now, remove that if you dont wanna test it
for index, match_id in enumerate(tqdm(manquants)):
    url = 'https://www.premierleague.com/match/' + str(match_id)
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')
    data = json.loads(soup.find('div', class_='mcTabsContainer').attrs['data-fixture'])

    players = []
    teams = []
    for team in data['teamLists']:
        team_id = team["teamId"]
    
        for player in team["lineup"]:
            player = flatten(player)
            player["substitute"] = "0"
            player["team_id"] = team_id
            players.append(flatten(player))

        for player in team["substitutes"]:
            player = flatten(player)
            player["substitute"] = "1"
            player["team_id"] = team_id
            players.append(flatten(player))
            
    playersdf = pd.DataFrame(players)
    playersdf['match_id'] = match_id
    df_players = df_players.append(playersdf)

    events = []
    for event in data['events']:
        events.append(flatten(event))
    eventsdf = pd.DataFrame(events)
    eventsdf['match_id'] = match_id
    df_events = df_events.append(eventsdf)

    data['teamLists'] = ''
    data['events'] = ''
    match = pd.DataFrame([flatten(data)])
    match['match_id'] = match_id
    df_games = df_games.append(match)
    
    
#     if index % 50 == 0:
    df_games.to_csv('real_data/all_games_92.csv', index=False)
    df_players.to_csv('real_data/all_players_92.csv', index=False)
    df_events.to_csv('real_data/all_events_92.csv', index=False)

/Applications/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Applications/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (68,69,70,71,72,81,82,83,84,85,88,89,90,91,92,95,96,97,98,99,101,103,104,105,106,107,110,111,112,113,114,117,118,119,120,121,124,125,126,127,128,131,132,133,134,135,138,139,140,141,142,145,146,147,148,149) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
  0%|          | 0/15 [00:00<?, ?it/s]

15


100%|██████████| 15/15 [02:46<00:00, 11.08s/it]


In [ ]:
len(set(df_match_ids["match_id"]) - set(pd.read_csv('real_data/all_games.csv', dtype={'match_id': 'str'})["match_id"]))

In [48]:
df_events.shape

(205567, 15)

In [ ]:
len(set(pd.read_csv('real_data/all_games.csv')["match_id"]))

In [ ]:
len(set(df_match_ids["match_id"]) - set(pd.read_csv('real_data/all_games.csv')["match_id"]))

In [ ]:
list(set(df_match_ids["match_id"]))[0], list(set(pd.read_csv('real_data/all_games.csv')["match_id"]))[0]